In [2]:
!pip install transformers datasets

In [ ]:
my_data = """Data"""



In [13]:
from datasets import Dataset
from transformers import GPT2Tokenizer

# Create a Dataset from the text data
dataset = Dataset.from_dict({"text": [my_data]})

# Load GPT-2 tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token  # GPT-2 has no pad token by default

# Tokenize dataset
def tokenize(example):
    tokenized_inputs = tokenizer(example["text"], padding="max_length", truncation=True, max_length=128)
    tokenized_inputs["labels"] = tokenized_inputs["input_ids"].copy() # Add labels for language modeling
    return tokenized_inputs

tokenized_dataset = dataset.map(tokenize, batched=True, remove_columns=["text"])

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

In [14]:
from transformers import GPT2LMHeadModel

model = GPT2LMHeadModel.from_pretrained("gpt2")
model.resize_token_embeddings(len(tokenizer))  # Adjust for any tokenizer change


Embedding(50257, 768)

In [15]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./gpt2_finetuned",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=2,
    save_steps=500,
    save_total_limit=2,
    logging_steps=50,
    logging_dir="./logs",
    prediction_loss_only=True,
    report_to="none",  # Disable wandb logging
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,  # Pass the dataset directly
    tokenizer=tokenizer,
)

/tmp/ipython-input-15-542638107.py:16: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [16]:
trainer.train()


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss


TrainOutput(global_step=3, training_loss=5.9647064208984375, metrics={'train_runtime': 53.1755, 'train_samples_per_second': 0.056, 'train_steps_per_second': 0.056, 'total_flos': 195969024000.0, 'train_loss': 5.9647064208984375, 'epoch': 3.0})

In [17]:
trainer.save_model("fine-tuned-gpt2")
tokenizer.save_pretrained("fine-tuned-gpt2")


('fine-tuned-gpt2/tokenizer_config.json',
 'fine-tuned-gpt2/special_tokens_map.json',
 'fine-tuned-gpt2/vocab.json',
 'fine-tuned-gpt2/merges.txt',
 'fine-tuned-gpt2/added_tokens.json')

In [18]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained("fine-tuned-gpt2")
model = GPT2LMHeadModel.from_pretrained("fine-tuned-gpt2")
model.eval()

prompt = "Once upon a time"
inputs = tokenizer(prompt, return_tensors="pt")
output = model.generate(
    **inputs,
    max_length=50,
    do_sample=True,
    top_p=0.9,
    temperature=0.8,
    no_repeat_ngram_size=2
)
print(tokenizer.decode(output[0], skip_special_tokens=True))


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Once upon a time, the game would be a slow game with a lot of action but the action is so great that you would have to just go and play with the player.

The player can also customize his skills with different items. He
